### Gerekli kütüphaneler <a id="2"> </a>

In [1]:
#Gerekli kütüphaneleri yüklemekle başlayalım. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn import preprocessing   
from warnings import filterwarnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

filterwarnings('ignore')



In [2]:
df = pd.read_csv("tablet.csv").copy()

### Örnek 5 veri

In [3]:
df.sample(5) 

,BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMP,4G,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMP,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,3G,Dokunmatik,WiFi,FiyatAraligi,Renk
826,1735,Yok,1.8,Var,16.0,Yok,34,0.1,140,2,17,149,876,2392.0,19,Var,Yok,Var,Normal,Siyah
883,1595,Var,0.8,Var,0.0,Var,44,0.7,85,4,20,1024,1838,3881.0,9,Var,Yok,Var,Pahalı,Mor
237,1829,Yok,1.7,Yok,0.0,Yok,36,0.8,112,5,1,291,563,1475.0,7,Yok,Yok,Yok,Ucuz,Kahverengi
1514,1163,Var,2.1,Var,0.0,Var,19,0.9,132,2,1,728,818,2940.0,15,Var,Yok,Var,Normal,Turkuaz
1230,1204,Var,2.6,Yok,0.0,Yok,10,0.3,199,7,3,480,1482,1724.0,19,Var,Yok,Yok,Ucuz,Turuncu


### Veri seti bilgileri

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BataryaGucu          2000 non-null   int64  
 1   Bluetooth            2000 non-null   object 
 2   MikroislemciHizi     2000 non-null   float64
 3   CiftHat              2000 non-null   object 
 4   OnKameraMP           1995 non-null   float64
 5   4G                   2000 non-null   object 
 6   DahiliBellek         2000 non-null   int64  
 7   Kalinlik             2000 non-null   float64
 8   Agirlik              2000 non-null   int64  
 9   CekirdekSayisi       2000 non-null   int64  
 10  ArkaKameraMP         2000 non-null   int64  
 11  CozunurlukYükseklik  2000 non-null   int64  
 12  CozunurlukGenislik   2000 non-null   int64  
 13  RAM                  1988 non-null   float64
 14  BataryaOmru          2000 non-null   int64  
 15  3G                   2000 non-null   o

In [5]:
print("Gözlem sayımız : ", df.shape[0])
print("Öznitelik sayımız: ",df.shape[1])

Gözlem sayımız :  2000
Öznitelik sayımız:  20


In [6]:
df.isnull().sum()

BataryaGucu             0
Bluetooth               0
MikroislemciHizi        0
CiftHat                 0
OnKameraMP              5
4G                      0
DahiliBellek            0
Kalinlik                0
Agirlik                 0
CekirdekSayisi          0
ArkaKameraMP            0
CozunurlukYükseklik     0
CozunurlukGenislik      0
RAM                    12
BataryaOmru             0
3G                      0
Dokunmatik              0
WiFi                    0
FiyatAraligi            0
Renk                    0
dtype: int64

In [7]:
df["Bluetooth"].unique()

array(['Yok', 'Var'], dtype=object)

In [8]:
df["CiftHat"].unique()

array(['Yok', 'Var'], dtype=object)

In [9]:
df["4G"].unique()

array(['Yok', 'Var'], dtype=object)

In [10]:
df["3G"].unique()

array(['Var', 'Yok'], dtype=object)

In [11]:
df["Dokunmatik"].unique()

array(['Yok', 'Var'], dtype=object)

In [12]:
df["WiFi"].unique()

array(['Yok', 'Var'], dtype=object)

In [13]:
df["FiyatAraligi"].unique()

array(['Normal', 'Pahalı', 'Ucuz', 'Çok Ucuz'], dtype=object)

In [14]:
df["Renk"].unique()

array(['Beyaz', 'Pembe', 'Mor', 'Turuncu', 'Gri', 'Sarı', 'Mavi',
       'Turkuaz', 'Kahverengi', 'Yeşil', 'Kırmızı', 'Siyah'], dtype=object)

##  Eksik Verilerin Tamamlanması <a id="9"> </a>


In [15]:
df.isnull().sum().sum()

17

In [16]:
df.isnull().sum() 

BataryaGucu             0
Bluetooth               0
MikroislemciHizi        0
CiftHat                 0
OnKameraMP              5
4G                      0
DahiliBellek            0
Kalinlik                0
Agirlik                 0
CekirdekSayisi          0
ArkaKameraMP            0
CozunurlukYükseklik     0
CozunurlukGenislik      0
RAM                    12
BataryaOmru             0
3G                      0
Dokunmatik              0
WiFi                    0
FiyatAraligi            0
Renk                    0
dtype: int64

Verisetimizin RAM özniteliği eksik olan 12 adet, ön kamera bilgisi girilmemiş olan 5 adet eksik gözlemi mevcut. 

In [17]:
df[df["OnKameraMP"].isnull()]

,BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMP,4G,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMP,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,3G,Dokunmatik,WiFi,FiyatAraligi,Renk
351,726,Yok,2.2,Yok,NaN,Var,40,0.8,94,4,20,323,978,1342.0,4,Var,Var,Var,Çok Ucuz,Beyaz
726,1296,Yok,0.9,Var,NaN,Var,14,0.5,124,5,3,811,1272,490.0,4,Var,Var,Var,Çok Ucuz,Sarı
792,504,Yok,2.8,Var,NaN,Yok,40,0.5,178,3,0,626,1195,470.0,16,Var,Yok,Yok,Çok Ucuz,Siyah
1416,1617,Var,2.4,Yok,NaN,Var,36,0.8,85,1,9,743,1426,296.0,7,Var,Yok,Yok,Çok Ucuz,Sarı
1641,763,Yok,1.7,Var,NaN,Var,2,0.5,92,8,14,1028,1186,1050.0,8,Var,Var,Yok,Çok Ucuz,Siyah


In [18]:
df.groupby("FiyatAraligi")["OnKameraMP"].mean()

FiyatAraligi
Normal      4.498000
Pahalı      4.316000
Ucuz        4.340000
Çok Ucuz    4.092929
Name: OnKameraMP, dtype: float64

In [19]:
NaN_onKamera = df[df["OnKameraMP"].isnull()].index
NaN_onKamera

Int64Index([351, 726, 792, 1416, 1641], dtype='int64')

In [20]:
df.loc[NaN_onKamera ,"OnKameraMP"] = 4 

In [21]:
df[df["RAM"].isnull()]

,BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMP,4G,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMP,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,3G,Dokunmatik,WiFi,FiyatAraligi,Renk
87,1062,Var,2.5,Var,6.0,Yok,10,0.9,160,1,13,1596,1807,NaN,16,Yok,Yok,Var,Pahalı,Gri
169,1944,Var,1.8,Var,0.0,Var,31,0.8,87,4,10,1111,1362,NaN,8,Var,Yok,Yok,Pahalı,Beyaz
310,1479,Var,1.8,Var,0.0,Var,24,0.1,146,6,0,327,645,NaN,5,Var,Yok,Yok,Pahalı,Turuncu
329,1035,Yok,0.6,Var,2.0,Var,44,0.5,99,3,3,236,1960,NaN,12,Var,Var,Yok,Pahalı,Beyaz
658,732,Var,1.3,Yok,5.0,Var,32,0.4,163,8,7,1661,1836,NaN,5,Var,Yok,Var,Pahalı,Siyah
777,753,Var,0.5,Yok,11.0,Var,48,0.1,121,8,20,606,1316,NaN,16,Var,Yok,Yok,Pahalı,Turkuaz
871,1035,Var,2.7,Yok,1.0,Yok,40,0.2,157,5,15,757,1912,NaN,5,Yok,Var,Var,Pahalı,Turuncu
961,1949,Yok,1.3,Var,0.0,Yok,23,0.1,101,5,1,839,1136,NaN,20,Var,Yok,Yok,Pahalı,Sarı
965,1786,Yok,1.2,Yok,11.0,Yok,61,0.1,123,3,14,74,865,NaN,17,Yok,Var,Yok,Pahalı,Kahverengi
1180,1369,Var,1.8,Var,6.0,Yok,34,0.5,162,1,7,553,727,NaN,18,Yok,Var,Var,Pahalı,Turuncu


In [22]:
df.groupby("FiyatAraligi")["RAM"].mean()

FiyatAraligi
Normal      2582.81600
Pahalı      3449.35041
Ucuz        1679.49000
Çok Ucuz     785.31400
Name: RAM, dtype: float64

In [23]:
NaN_Ram= df[df["RAM"].isnull()].index
NaN_Ram

Int64Index([87, 169, 310, 329, 658, 777, 871, 961, 965, 1180, 1779, 1804], dtype='int64')

In [24]:
df.loc[NaN_Ram ,"RAM"] = 3450 

In [25]:
df.isnull().sum().sum()

0

### Kategorik değerlere dönüştürme   <a id="11"> </a>

In [26]:
sns.catplot(x="FiyatAraligi", y="Renk",data = df)

NameError: name 'sns' is not defined

Renklerin fiyat aralığına bir etkisi olmadığını gördük. Bu yüzden renk özniteliğini çıkartabiliriz.

In [ ]:
df.drop('Renk', inplace=True, axis=1)

In [27]:
label_encoder = preprocessing.LabelEncoder()
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth'])
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat'])
df['4G'] = label_encoder.fit_transform(df['4G'])
df['3G'] = label_encoder.fit_transform(df['3G'])
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik'])
df['WiFi'] = label_encoder.fit_transform(df['WiFi'])
df['Renk'] = label_encoder.fit_transform(df['Renk'])
df['FiyatAraligi'] = label_encoder.fit_transform(df['FiyatAraligi'])

df.head()

,BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMP,4G,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMP,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,3G,Dokunmatik,WiFi,FiyatAraligi,Renk
0,1325,1,1.9,1,2.0,1,50,0.1,146,1,10,499,695,2328.0,2,0,1,1,0,0
1,1046,0,2.8,0,0.0,0,58,0.2,100,8,0,259,1040,3863.0,9,0,0,0,1,6
2,843,1,1.3,0,1.0,1,16,0.9,119,7,4,593,732,1734.0,14,1,1,0,2,5
3,894,1,0.5,0,1.0,0,50,0.5,199,3,14,1285,1462,3615.0,6,0,1,0,1,10
4,936,1,1.0,1,1.0,1,18,0.2,153,3,18,1330,1686,2391.0,3,0,0,0,0,0


## Modellerin oluşturulması ve eğitilmesi <a id="12"> </a>

In [42]:
y = df['FiyatAraligi']
X = df.drop(['FiyatAraligi'], axis=1)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)
                                                    

### Desicion tree <a id="14"> </a>

In [44]:
cart = DecisionTreeClassifier(random_state = 0)
cart_model = cart.fit(X_train, y_train)

In [45]:
y_pred = cart_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.818

In [46]:
importance = cart_model.feature_importances_
for i,v in enumerate(importance):
    print('Öznitelik: %0d, Skor: %.5f' % (i,v))


Öznitelik: 0, Skor: 0.11739
Öznitelik: 1, Skor: 0.00450
Öznitelik: 2, Skor: 0.00705
Öznitelik: 3, Skor: 0.00089
Öznitelik: 4, Skor: 0.00954
Öznitelik: 5, Skor: 0.00089
Öznitelik: 6, Skor: 0.00898
Öznitelik: 7, Skor: 0.00980
Öznitelik: 8, Skor: 0.01539
Öznitelik: 9, Skor: 0.00551
Öznitelik: 10, Skor: 0.00840
Öznitelik: 11, Skor: 0.07387
Öznitelik: 12, Skor: 0.07097
Öznitelik: 13, Skor: 0.64845
Öznitelik: 14, Skor: 0.00747
Öznitelik: 15, Skor: 0.00207
Öznitelik: 16, Skor: 0.00119
Öznitelik: 17, Skor: 0.00000
Öznitelik: 18, Skor: 0.00764


In [49]:
X = df[['BataryaGucu', 'CozunurlukGenislik', 'RAM']]
y = df['FiyatAraligi']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

In [50]:
cart = DecisionTreeClassifier(random_state = 0)
cart_model = cart.fit(X_train, y_train)

In [51]:
y_pred = cart_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.838

In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79       131
           1       0.87      0.81      0.84       113
           2       0.83      0.80      0.81       127
           3       0.90      0.93      0.91       129

    accuracy                           0.84       500
   macro avg       0.84      0.84      0.84       500
weighted avg       0.84      0.84      0.84       500



In [53]:
param_grid = {'max_depth': [3, 5, 7, 10],
              'min_samples_split': [2, 5, 10, 20]}

# GridSearchCV ile hipertarametre arama ve en iyi parametreleri bulma
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# En iyi parametreleri kullanarak final modelini oluşturma
best_params = grid_search.best_params_
final_model = DecisionTreeClassifier(max_depth=best_params['max_depth'], 
                                      min_samples_split=best_params['min_samples_split'])

# Final modeli eğitme
final_model.fit(X, y)

DecisionTreeClassifier(max_depth=7, min_samples_split=20)

In [54]:
y_pred = final_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.916

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       131
           1       0.93      0.93      0.93       113
           2       0.92      0.87      0.90       127
           3       0.93      0.98      0.95       129

    accuracy                           0.92       500
   macro avg       0.92      0.92      0.92       500
weighted avg       0.92      0.92      0.92       500



In [57]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)
cross_val_score(cart_model, X_test, y_test, cv = 10).mean()

[[116   8   7   0]
 [  8 105   0   0]
 [  6   0 111  10]
 [  0   0   3 126]]


0.8140000000000001

Normal -> 0

Pahalı -> 1

Ucuz -> 2

Çok Ucuz -> 3

Yok -> 1
Var -> 0

In [ ]:
BataryaGucu = 2500
Bluetooth = 1
MikroislemciHizi = 2.2
CiftHat = 0
OnKameraMp = 5
DortG = 0
DahiliBellek = 32
Kalinlik = 0.9
Agirlik = 120
CekirdekSayisi = 4
ArkaKameraMp = 12
CozunurlukYükseklik = 900
CozunurlukGenislik = 1200
RAM = 3000
BataryaOmru = 12
UcG = 0
Dokunmatik = 0
WiFi = 0
cart_model.predict([[BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMp,DortG,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMp,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,UcG,Dokunmatik,WiFi]])

In [ ]:
test = pd.DataFrame(X_test).copy() 
test["fiyatAraligi"] = y_test
test["predicted_fiyatAraligi"] = y_pred
test.sample(5)

### KNN <a id="15"> </a>

In [93]:
y = df['FiyatAraligi']
X = df.drop(['FiyatAraligi'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

In [83]:
# KNN sınıflandırıcı nesnesi oluştur
knn = KNeighborsClassifier()

# GridSearchCV nesnesi oluştur ve parametreleri ayarla
param_grid = {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean', 'manhattan']}
grid_search = GridSearchCV(knn, param_grid, cv=5)

# GridSearchCV nesnesini eğit
grid_search.fit(X_train, y_train)

# En iyi parametreleri yazdır
print("En iyi parametreler: ", grid_search.best_params_)

En iyi parametreler:  {'metric': 'euclidean', 'n_neighbors': 9}


In [88]:
knn_tuned = KNeighborsClassifier(metric = "euclidean", n_neighbors = 9)
knn_tuned = knn_tuned.fit(X_train, y_train)
y_pred = knn_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.934

In [ ]:
BataryaGucu = 2500
Bluetooth = 1
MikroislemciHizi = 2.2
CiftHat = 0
OnKameraMp = 5
DortG = 0
DahiliBellek = 32
Kalinlik = 0.9
Agirlik = 120
CekirdekSayisi = 4
ArkaKameraMp = 12
CozunurlukYükseklik = 900
CozunurlukGenislik = 1200
RAM = 3000
BataryaOmru = 12
UcG = 0
Dokunmatik = 0
WiFi = 0
knn_tuned.predict([[BataryaGucu,Bluetooth,MikroislemciHizi,CiftHat,OnKameraMp,DortG,DahiliBellek,Kalinlik,Agirlik,CekirdekSayisi,ArkaKameraMp,CozunurlukYükseklik,CozunurlukGenislik,RAM,BataryaOmru,UcG,Dokunmatik,WiFi]])

In [87]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)
cross_val_score(knn_tuned, X_test, y_test, cv = 10).mean()

[[116   7   8   0]
 [  3 110   0   0]
 [  4   0 116   7]
 [  0   0   4 125]]


0.906

In [86]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91       131
           1       0.94      0.97      0.96       113
           2       0.91      0.91      0.91       127
           3       0.95      0.97      0.96       129

    accuracy                           0.93       500
   macro avg       0.93      0.94      0.93       500
weighted avg       0.93      0.93      0.93       500



CHATGPT results


In [95]:


# Model Seçimi
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Hiperparametre Ayarı
from sklearn.model_selection import RandomizedSearchCV

params = {'n_neighbors': [3, 5, 7, 9, 11],
          'weights': ['uniform', 'distance'],
          'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
          'p': [1, 2]}
          
knn = KNeighborsClassifier()
knn_random = RandomizedSearchCV(estimator=knn, param_distributions=params, n_iter=100, cv=5, random_state=42)
knn_random.fit(X_train, y_train)
y_pred = knn_random.predict(X_test)
accuracy_score(y_test, y_pred)

0.938

In [96]:

# Veri Artırma
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Model Ensemble
from sklearn.ensemble import VotingClassifier

dt = DecisionTreeClassifier()
svm = SVC(probability=True)

voting = VotingClassifier(estimators=[('dt', dt), ('knn', knn), ('svm', svm)], voting='soft')
voting.fit(X_train, y_train)

NameError: name 'SVC' is not defined